# Комп'ютерний практикум №1: Метод кореляційного аналізу даних

**Дисципліна:** Інтелектуальний аналіз даних  
**Тема:** Кореляційний аналіз  
**Виконав:** [ПІБ студента]  
**Група:** [Номер групи]  
**Викладач:** [ПІБ викладача]


## 1. Параметри та налаштування


In [ ]:
# Параметри аналізу
user_data_path = None  # або 'path/to/data.csv'
alpha = 0.05  # рівень значущості
random_state = 42  # для відтворюваності результатів


## 2. Імпорт бібліотек


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import shapiro, kstest, pearsonr, spearmanr
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Встановлення seed для відтворюваності
np.random.seed(random_state)

# Створення папки для результатів
output_dir = Path('outputs')
output_dir.mkdir(exist_ok=True)

print("[OK] Всі бібліотеки успішно імпортовано")
print(f"[OK] Папка для результатів: {output_dir.absolute()}")


## 3. Мета роботи

Оволодіння методами кореляційного аналізу для дослідження статистичних зв'язків між змінними, визначення сили та напрямку зв'язку, перевірки статистичної значущості кореляційних коефіцієнтів.


## 4. Завдання

1. Оволодіти методами кореляційного аналізу (коефіцієнти Пірсона та Спірмена).
2. Виконати кореляційний аналіз між змінними, визначити силу та напрямок зв'язку, перевірити статистичну значущість.


## 5. Генерація або завантаження даних


In [ ]:
def generate_synthetic_data(n_samples=200, missing_rate=0.07):
    """
    Генерація синтетичних даних з правдоподібними кореляціями
    
    Очікувані зв'язки:
    - sleep_hours ↗ productivity_score (помірно позитивна)
    - steps_per_day ↘ weight_kg (слабко/помірно негативна)
    - calories_intake ↗ weight_kg (позитивна)
    - caffeine_mg ↗ productivity_score (слабко позитивна)
    """
    np.random.seed(random_state)
    
    # Базові незалежні змінні
    sleep_hours = np.random.normal(7, 1.2, n_samples)
    sleep_hours = np.clip(sleep_hours, 4, 10)
    
    steps_per_day = np.random.normal(8000, 2500, n_samples)
    steps_per_day = np.clip(steps_per_day, 2000, 15000)
    
    caffeine_mg = np.random.normal(150, 80, n_samples)
    caffeine_mg = np.clip(caffeine_mg, 0, 400)
    
    calories_intake = np.random.normal(2000, 400, n_samples)
    calories_intake = np.clip(calories_intake, 1200, 3500)
    
    # Залежні змінні з кореляціями
    # productivity_score: залежить від сну (+) та кофеїну (+)
    productivity_score = (
        30 + 
        6 * sleep_hours +  # помірна позитивна кореляція
        0.03 * caffeine_mg +  # слабка позитивна кореляція
        np.random.normal(0, 8, n_samples)
    )
    productivity_score = np.clip(productivity_score, 0, 100)
    
    # weight_kg: залежить від калорій (+) та кроків (-)
    weight_kg = (
        40 + 
        0.012 * calories_intake +  # позитивна кореляція
        -0.0015 * steps_per_day +  # негативна кореляція
        np.random.normal(0, 5, n_samples)
    )
    weight_kg = np.clip(weight_kg, 50, 100)
    
    # Створення DataFrame
    df = pd.DataFrame({
        'sleep_hours': sleep_hours,
        'productivity_score': productivity_score,
        'steps_per_day': steps_per_day,
        'calories_intake': calories_intake,
        'weight_kg': weight_kg,
        'caffeine_mg': caffeine_mg
    })
    
    # Додавання пропусків (5-10%)
    n_missing = int(n_samples * len(df.columns) * missing_rate)
    missing_indices = np.random.choice(
        n_samples * len(df.columns), 
        size=n_missing, 
        replace=False
    )
    
    for idx in missing_indices:
        row = idx // len(df.columns)
        col = idx % len(df.columns)
        df.iloc[row, col] = np.nan
    
    return df


# Завантаження або генерація даних
if user_data_path is not None:
    print(f"Завантаження даних з файлу: {user_data_path}")
    df = pd.read_csv(user_data_path)
    print("[OK] Дані завантажено з CSV")
else:
    print("Генерація синтетичних даних...")
    df = generate_synthetic_data(n_samples=200, missing_rate=0.07)
    
    # Збереження синтетичних даних
    synthetic_path = output_dir / 'synthetic_data.csv'
    df.to_csv(synthetic_path, index=False)
    print(f"[OK] Синтетичні дані збережено: {synthetic_path}")

print(f"\nРозмір датасету: {df.shape[0]} рядків x {df.shape[1]} стовпців")
df.head(10)


In [ ]:
print("=" * 60)
print("ІНФОРМАЦІЯ ПРО ДАТАСЕТ")
print("=" * 60)
df.info()
print("\n" + "=" * 60)


### 6.2. Описова статистика


In [ ]:
print("ОПИСОВА СТАТИСТИКА")
print("=" * 60)
df.describe().round(2)


### 6.3. Аналіз пропущених значень


In [ ]:
# Розрахунок пропусків
missing_data = pd.DataFrame({
    'Змінна': df.columns,
    'Пропусків': df.isnull().sum(),
    'Частка (%)': (df.isnull().sum() / len(df) * 100).round(2)
})

print("АНАЛІЗ ПРОПУЩЕНИХ ЗНАЧЕНЬ")
print("=" * 60)
print(missing_data.to_string(index=False))
print("\nЗагальна частка пропусків: {:.2f}%".format(
    df.isnull().sum().sum() / (len(df) * len(df.columns)) * 100
))
print("=" * 60)


### 6.4. Обробка пропусків: два підходи

**Підхід A (Pairwise):** Попарне виключення - використовуємо всі доступні дані для кожної пари змінних.

**Підхід B (Listwise):** Повне виключення - видаляємо всі рядки з будь-якими пропусками.


In [ ]:
# Підхід A: Pairwise (оригінальні дані з пропусками)
df_pairwise = df.copy()

# Підхід B: Listwise (видалення рядків з пропусками)
df_listwise = df.dropna()

print("ПОРІВНЯННЯ ПІДХОДІВ ДО ОБРОБКИ ПРОПУСКІВ")
print("=" * 60)
print(f"Оригінальний датасет: {len(df)} спостережень")
print(f"Після pairwise: {len(df_pairwise)} спостережень (без змін)")
print(f"Після listwise: {len(df_listwise)} спостережень (-{len(df) - len(df_listwise)} рядків)")
print("=" * 60)


## 7. Перевірка нормальності розподілу

### 7.1. Функція для тестування нормальності


In [ ]:
def test_normality(data, var_name, alpha=0.05):
    """
    Перевірка нормальності розподілу змінної
    
    Використовує:
    - Shapiro-Wilk тест (якщо n ≤ 5000)
    - Kolmogorov-Smirnov тест (якщо n > 5000)
    
    H0: дані мають нормальний розподіл
    H1: дані не мають нормального розподілу
    """
    # Видалення пропусків
    clean_data = data.dropna()
    n = len(clean_data)
    
    if n == 0:
        return {
            'variable': var_name,
            'test': 'N/A',
            'statistic': np.nan,
            'p_value': np.nan,
            'n': 0,
            'normality_conclusion': 'Недостатньо даних'
        }
    
    # Вибір тесту
    if n <= 5000:
        test_name = 'Shapiro-Wilk'
        statistic, p_value = shapiro(clean_data)
    else:
        test_name = 'Kolmogorov-Smirnov'
        # KS тест проти нормального розподілу з параметрами вибірки
        mean, std = clean_data.mean(), clean_data.std()
        statistic, p_value = kstest(clean_data, lambda x: stats.norm.cdf(x, mean, std))
    
    # Висновок
    if p_value >= alpha:
        conclusion = f'Нормальний (p={p_value:.4f} ≥ {alpha})'
        is_normal = True
    else:
        conclusion = f'Не нормальний (p={p_value:.4f} < {alpha})'
        is_normal = False
    
    return {
        'variable': var_name,
        'test': test_name,
        'statistic': statistic,
        'p_value': p_value,
        'n': n,
        'normality_conclusion': conclusion,
        'is_normal': is_normal
    }

print("[OK] Функція тестування нормальності створена")


In [ ]:
# Тестування для listwise підходу
normality_results = []

for col in df_listwise.select_dtypes(include=[np.number]).columns:
    result = test_normality(df_listwise[col], col, alpha)
    normality_results.append(result)

# Створення таблиці результатів
normality_df = pd.DataFrame(normality_results)
normality_df_display = normality_df[['variable', 'test', 'statistic', 'p_value', 'n', 'normality_conclusion']]

# Збереження результатів
normality_path = output_dir / 'normality_summary.csv'
normality_df_display.to_csv(normality_path, index=False)

print("РЕЗУЛЬТАТИ ТЕСТІВ НОРМАЛЬНОСТІ")
print("=" * 80)
print(normality_df_display.to_string(index=False))
print("\n" + "=" * 80)
print(f"[OK] Результати збережено: {normality_path}")

# Створення словника для швидкого доступу
normality_dict = {row['variable']: row['is_normal'] for _, row in normality_df.iterrows()}


## 8. Вибір методу кореляції та інтерпретація

### 8.1. Методика вибору коефіцієнта кореляції

**Правила вибору:**
- Якщо обидві змінні мають нормальний розподіл → **Коефіцієнт Пірсона** (параметричний)
- Якщо хоча б одна змінна не має нормального розподілу → **Коефіцієнт Спірмена** (непараметричний, ранговий)


In [ ]:
def interpret_strength(r):
    """
    Інтерпретація сили зв'язку за шкалою з методички
    
    Шкала:
    0.75-1.00: дуже високий
    0.50-0.74: високий
    0.25-0.49: середній
    0.00-0.24: слабкий
    """
    abs_r = abs(r)
    direction = "позитивний" if r >= 0 else "негативний"
    
    if abs_r >= 0.75:
        strength = "дуже високий"
    elif abs_r >= 0.50:
        strength = "високий"
    elif abs_r >= 0.25:
        strength = "середній"
    else:
        strength = "слабкий"
    
    return f"{strength} {direction}"


def choose_correlation_method(x, y, var_x, var_y, normality_dict):
    """
    Вибір методу кореляції на основі нормальності змінних
    
    Повертає: method, coefficient, p_value, n
    """
    # Видалення пропусків для пари
    mask = ~(pd.isna(x) | pd.isna(y))
    x_clean = x[mask]
    y_clean = y[mask]
    n = len(x_clean)
    
    if n < 3:
        return 'N/A', np.nan, np.nan, n
    
    # Перевірка нормальності
    x_normal = normality_dict.get(var_x, False)
    y_normal = normality_dict.get(var_y, False)
    
    # Вибір методу
    if x_normal and y_normal:
        method = 'Pearson'
        coef, p_val = pearsonr(x_clean, y_clean)
    else:
        method = 'Spearman'
        coef, p_val = spearmanr(x_clean, y_clean)
    
    return method, coef, p_val, n

print("[OK] Функції вибору методу та інтерпретації створено")


## 9. Розрахунок кореляцій

### 9.1. Функція для обчислення всіх кореляцій


In [ ]:
def calculate_all_correlations(df, normality_dict, approach_name, alpha=0.05):
    """
    Обчислення кореляцій для всіх пар змінних
    
    Параметри:
    - df: DataFrame з даними
    - normality_dict: словник з результатами тестів нормальності
    - approach_name: 'pairwise' або 'listwise'
    - alpha: рівень значущості
    """
    results = []
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    
    for i, var_x in enumerate(numeric_cols):
        for var_y in numeric_cols[i+1:]:
            method, coef, p_val, n = choose_correlation_method(
                df[var_x], df[var_y], var_x, var_y, normality_dict
            )
            
            if not np.isnan(coef):
                # Визначення значущості
                if p_val < 0.01:
                    significance = '**'
                elif p_val < alpha:
                    significance = '*'
                else:
                    significance = ''
                
                # Висновок про гіпотезу
                h0_conclusion = 'Відхилено' if p_val < alpha else 'Не відхилено'
                
                results.append({
                    'variable_x': var_x,
                    'variable_y': var_y,
                    'method': method,
                    'coefficient': coef,
                    'p_value': p_val,
                    'n': n,
                    'significance': significance,
                    'strength_label': interpret_strength(coef),
                    'H0_conclusion': h0_conclusion,
                    'approach': approach_name
                })
    
    return pd.DataFrame(results)

print("[OK] Функція обчислення кореляцій створена")


### 9.2. Обчислення кореляцій для обох підходів


In [ ]:
# Підхід A: Pairwise
print("Обчислення кореляцій (Pairwise)...")
corr_pairwise = calculate_all_correlations(df_pairwise, normality_dict, 'pairwise', alpha)
corr_pairwise_path = output_dir / 'correlations_pairwise.csv'
corr_pairwise.to_csv(corr_pairwise_path, index=False)
print(f"[OK] Збережено: {corr_pairwise_path}")

# Підхід B: Listwise
print("Обчислення кореляцій (Listwise)...")
corr_listwise = calculate_all_correlations(df_listwise, normality_dict, 'listwise', alpha)
corr_listwise_path = output_dir / 'correlations_listwise.csv'
corr_listwise.to_csv(corr_listwise_path, index=False)
print(f"[OK] Збережено: {corr_listwise_path}")

print("\n" + "=" * 100)
print("РЕЗУЛЬТАТИ КОРЕЛЯЦІЙНОГО АНАЛІЗУ (PAIRWISE)")
print("=" * 100)
display_cols = ['variable_x', 'variable_y', 'method', 'coefficient', 'p_value', 'n', 'significance', 'strength_label']
print(corr_pairwise[display_cols].to_string(index=False))
print("\n" + "=" * 100)


### 9.3. Таблиця кореляцій у стилі SPSS для основної пари


In [ ]:
# Вибираємо основну пару: sleep_hours vs productivity_score
main_pair = corr_listwise[
    (corr_listwise['variable_x'] == 'sleep_hours') & 
    (corr_listwise['variable_y'] == 'productivity_score')
]

if len(main_pair) > 0:
    row = main_pair.iloc[0]
    
    print("\n" + "=" * 80)
    print("ТАБЛИЦЯ КОРЕЛЯЦІЙ (СТИЛЬ SPSS)")
    print("Основна пара: sleep_hours ↔ productivity_score")
    print("=" * 80)
    print(f"\nМетод: {row['method']}")
    print(f"Коефіцієнт кореляції (r): {row['coefficient']:.4f}{row['significance']}")
    print(f"P-значення (двостороннє): {row['p_value']:.4f}")
    print(f"Кількість спостережень (N): {row['n']}")
    print(f"Інтерпретація: {row['strength_label']}")
    print(f"\nПримітки:")
    print(f"  * Кореляція значуща на рівні 0.05 (двостороння)")
    print(f"  ** Кореляція значуща на рівні 0.01 (двостороння)")
    print("=" * 80)
else:
    print("Основна пара не знайдена в результатах")


## 10. Візуалізація результатів

### 10.1. Діаграми розсіювання для ключових пар


In [ ]:
def create_scatter_plot(df, x_var, y_var, method, r, p_val, n, output_path):
    """
    Створення діаграми розсіювання з лінією регресії (для Пірсона)
    """
    # Очищення даних
    mask = ~(pd.isna(df[x_var]) | pd.isna(df[y_var]))
    x_clean = df[x_var][mask]
    y_clean = df[y_var][mask]
    
    # Створення графіка
    plt.figure(figsize=(8, 6))
    plt.scatter(x_clean, y_clean)
    
    # Лінія регресії тільки для Пірсона
    if method == 'Pearson':
        z = np.polyfit(x_clean, y_clean, 1)
        p = np.poly1d(z)
        plt.plot(x_clean, p(x_clean), linestyle='--')
    
    plt.xlabel(x_var.replace('_', ' ').title())
    plt.ylabel(y_var.replace('_', ' ').title())
    plt.title(f'{x_var} vs {y_var}
{method}: r={r:.3f}, p={p_val:.4f}, n={n}')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    return output_path

# Вибір 4 цікавих пар для візуалізації
pairs_to_plot = [
    ('sleep_hours', 'productivity_score'),
    ('steps_per_day', 'weight_kg'),
    ('calories_intake', 'weight_kg'),
    ('caffeine_mg', 'productivity_score')
]

scatter_plots = []

print("Створення діаграм розсіювання...
")
for x_var, y_var in pairs_to_plot:
    # Знаходимо результати для цієї пари
    pair_result = corr_listwise[
        ((corr_listwise['variable_x'] == x_var) & (corr_listwise['variable_y'] == y_var)) |
        ((corr_listwise['variable_x'] == y_var) & (corr_listwise['variable_y'] == x_var))
    ]
    
    if len(pair_result) > 0:
        row = pair_result.iloc[0]
        output_path = output_dir / f'scatter_{x_var}_vs_{y_var}.png'
        create_scatter_plot(
            df_listwise, x_var, y_var, 
            row['method'], row['coefficient'], row['p_value'], row['n'],
            output_path
        )
        scatter_plots.append(output_path)
        print(f"[OK] {output_path.name}")

print(f"
[OK] Створено {len(scatter_plots)} діаграм розсіювання")


### 10.2. Матриця кореляцій


In [ ]:
# Створення матриці кореляцій Пірсона для візуалізації
corr_matrix = df_listwise.corr(method='pearson')

# Візуалізація через matplotlib
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(corr_matrix, cmap='coolwarm', aspect='auto', vmin=-1, vmax=1)

# Налаштування осей
ax.set_xticks(np.arange(len(corr_matrix.columns)))
ax.set_yticks(np.arange(len(corr_matrix.columns)))
ax.set_xticklabels(corr_matrix.columns, rotation=45, ha='right')
ax.set_yticklabels(corr_matrix.columns)

# Додавання значень кореляцій
for i in range(len(corr_matrix.columns)):
    for j in range(len(corr_matrix.columns)):
        text = ax.text(j, i, f'{corr_matrix.iloc[i, j]:.2f}',
                      ha="center", va="center", color="black", fontsize=9)

# Colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Коефіцієнт кореляції', rotation=270, labelpad=20)

plt.title('Матриця кореляцій (Pearson, listwise)', pad=20)
plt.tight_layout()

# Збереження
matrix_path = output_dir / 'correlation_matrix.png'
plt.savefig(matrix_path, dpi=300, bbox_inches='tight')
plt.close()

print(f"[OK] Матриця кореляцій збережена: {matrix_path.name}")

# Також збережемо як CSV
matrix_csv_path = output_dir / 'correlation_matrix.csv'
corr_matrix.to_csv(matrix_csv_path)
print(f"[OK] Матриця кореляцій (CSV): {matrix_csv_path.name}")


## 11. Висновок по основній парі змінних


In [ ]:
# Формування висновку для основної пари
if len(main_pair) > 0:
    row = main_pair.iloc[0]
    
    conclusion_text = f"""
ВИСНОВОК ПО ОСНОВНІЙ ПАРІ ЗМІННИХ
{'=' * 80}

Змінні: {row['variable_x']} ↔ {row['variable_y']}

Результати аналізу:
  • Метод: {row['method']} (обрано на основі тестів нормальності)
  • Коефіцієнт кореляції: r = {row['coefficient']:.4f}
  • P-значення: p = {row['p_value']:.4f}
  • Кількість спостережень: n = {row['n']}
  • Сила зв'язку: {row['strength_label']}

Статистичний висновок:
На рівні значущості α = {alpha}, гіпотезу H₀: ρ = 0 (відсутність кореляції) 
{'ВІДХИЛЕНО' if row['p_value'] < alpha else 'НЕ ВІДХИЛЕНО'}.

Інтерпретація:
Було виявлено {row['strength_label']} зв'язок між кількістю годин сну та 
показником продуктивності (r = {row['coefficient']:.4f}, p = {row['p_value']:.4f}, n = {row['n']}).

{'Це означає, що зі збільшенням тривалості сну спостерігається тенденція до' if row['coefficient'] > 0 else 'Це означає, що зі збільшенням тривалості сну спостерігається тенденція до зниження'}
{'підвищення' if row['coefficient'] > 0 else ''} продуктивності. Зв'язок є статистично значущим,
що свідчить про наявність реальної асоціації між цими змінними в досліджуваній
популяції.

{'=' * 80}
"""
    
    print(conclusion_text)
    
    # Збереження висновку
    conclusion_path = output_dir / 'main_pair_conclusion.txt'
    with open(conclusion_path, 'w', encoding='utf-8') as f:
        f.write(conclusion_text)
    
    print(f"
[OK] Висновок збережено: {conclusion_path}")


## 12. Автоматична генерація звіту


In [ ]:
def generate_markdown_report():
    """Генерація markdown звіту згідно з вимогами методички"""
    
    report = f"""# Звіт про виконання комп'ютерного практикуму №1
## Метод кореляційного аналізу даних

---

## 1. ВСТУПНА ЧАСТИНА

**ВНЗ:** [Назва навчального закладу]  
**Кафедра:** [Назва кафедри]  
**Дисципліна:** Інтелектуальний аналіз даних  
**№ роботи:** 1  
**Тема:** Метод кореляційного аналізу даних  
**Група:** [Номер групи]  
**Виконав:** [ПІБ студента]  
**Викладач:** [ПІБ викладача]  

---

## 2. МЕТА РОБОТИ

Оволодіння методами кореляційного аналізу для дослідження статистичних зв'язків між змінними, визначення сили та напрямку зв'язку, перевірки статистичної значущості кореляційних коефіцієнтів.

---

## 3. ЗАВДАННЯ

1. Оволодіти методами кореляційного аналізу (коефіцієнти Пірсона та Спірмена).
2. Виконати кореляційний аналіз між змінними, визначити силу та напрямок зв'язку, перевірити статистичну значущість.

---

## 4. МЕТОДИКА

### 4.1. Вибір методу кореляції

Для кожної пари змінних метод кореляції обирався за наступними правилами:

- **Коефіцієнт Пірсона** (параметричний): якщо обидві змінні мають нормальний розподіл
- **Коефіцієнт Спірмена** (непараметричний, ранговий): якщо хоча б одна змінна не має нормального розподілу

### 4.2. Перевірка нормальності

Для перевірки нормальності розподілу використовувався:
- **Тест Shapiro-Wilk** (для вибірок n ≤ 5000)
- **Тест Kolmogorov-Smirnov** (для вибірок n > 5000)

Гіпотеза H₀: дані мають нормальний розподіл.  
Рівень значущості: α = {alpha}

### 4.3. Обробка пропущених значень

Використано два підходи:

- **Підхід A (Pairwise deletion):** Попарне виключення
- **Підхід B (Listwise deletion):** Повне виключення

### 4.4. Інтерпретація сили зв'язку

| |r| | Інтерпретація |
|------|---------------|
| 0.75-1.00 | Дуже високий |
| 0.50-0.74 | Високий |
| 0.25-0.49 | Середній |
| 0.00-0.24 | Слабкий |

---

## 5. РЕЗУЛЬТАТИ

### 5.1. Характеристика даних

- **Кількість спостережень (pairwise):** {len(df_pairwise)}
- **Кількість спостережень (listwise):** {len(df_listwise)}
- **Кількість змінних:** {len(df.columns)}
- **Змінні:** {', '.join(df.columns.tolist())}

### 5.2. Результати кореляційного аналізу

#### Топ-5 найсильніших кореляцій (Listwise):

"""
    
    # Додаємо топ кореляції
    top_corr = corr_listwise.nlargest(5, 'coefficient', keep='all')[['variable_x', 'variable_y', 'method', 'coefficient', 'p_value', 'strength_label']]
    report += "\n" + top_corr.to_string(index=False) + "\n\n"
    
    report += f"""Повні результати збережено у файлах:
- `correlations_pairwise.csv`
- `correlations_listwise.csv`

### 5.3. Візуалізації

Створено наступні графіки:

1. **Діаграми розсіювання**
2. **Матриця кореляцій:** `correlation_matrix.png`

---

## 6. ВИСНОВКИ

"""
    
    if len(main_pair) > 0:
        row = main_pair.iloc[0]
        report += f"""1. **Основний результат:** Виявлено {row['strength_label']} зв'язок між {row['variable_x']} та {row['variable_y']} (r = {row['coefficient']:.4f}, p = {row['p_value']:.4f}).

2. **Метод аналізу:** На основі тестів нормальності для аналізу використовувався коефіцієнт {row['method']}.

3. **Статистична значущість:** Кореляція є {'статистично значущою' if row['p_value'] < alpha else 'статистично незначущою'} на рівні α = {alpha}.

4. **Обробка пропусків:** Використано два підходи до обробки пропущених значень.

5. **Практичне значення:** Отримані результати мають практичне значення для розуміння взаємозв'язків між змінними.

"""

    report += """---

## 7. ВІДПОВІДІ НА КОНТРОЛЬНІ ЗАПИТАННЯ

### 7.1. Які задачі вирішує кореляційний аналіз?

- Визначення наявності статистичного зв'язку між змінними
- Оцінка сили та напрямку зв'язку
- Перевірка статистичної значущості виявлених зв'язків
- Відбір змінних для регресійного аналізу
- Виявлення мультиколінеарності

### 7.2. Як знайти коефіцієнт кореляції?

**Коефіцієнт Пірсона:** r = Σ[(xi - x̄)(yi - ȳ)] / √[Σ(xi - x̄)² x Σ(yi - ȳ)²]

**Коефіцієнт Спірмена:** ρ = 1 - [6Σdi²] / [n(n² - 1)]

### 7.3. Які типи коефіцієнтів кореляції існують?

1. **Пірсона** - параметричний метод для лінійних зв'язків
2. **Спірмена** - непараметричний ранговий метод
3. **Кендалла** - непараметричний метод на основі конкордантності
4. **Точковий бісеріальний** - для зв'язку безперервної та бінарної змінних
5. **Фі-коефіцієнт** - для двох бінарних змінних

### 7.4. Різниця між прямою та зворотною кореляцією?

- **Пряма кореляція** (r > 0): Зі збільшенням однієї змінної збільшується інша
- **Зворотна кореляція** (r < 0): Зі збільшенням однієї змінної зменшується інша
- **Відсутність кореляції** (r ≈ 0): Змінні не пов'язані лінійно

### 7.5. Як перевіряється значущість коефіцієнта кореляції?

1. **Гіпотези:** H₀: ρ = 0 vs H₁: ρ ≠ 0
2. **t-статистика:** t = r√(n-2) / √(1-r²)
3. **Рішення:** Якщо p < α, відхиляємо H₀

---

## 8. СПИСОК ЗГЕНЕРОВАНИХ ФАЙЛІВ

- `synthetic_data.csv`
- `normality_summary.csv`
- `correlations_pairwise.csv`
- `correlations_listwise.csv`
- `correlation_matrix.csv`
- `correlation_matrix.png`
- Діаграми розсіювання (4 файли)
- `main_pair_conclusion.txt`
- `report_draft.md`

---

**Дата виконання:** {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}
"""
    
    return report

# Генерація звіту
report_content = generate_markdown_report()
report_path = output_dir / 'report_draft.md'

with open(report_path, 'w', encoding='utf-8') as f:
    f.write(report_content)

print("=" * 80)
print("ЗВІТ УСПІШНО ЗГЕНЕРОВАНО")
print("=" * 80)
print(f"[OK] Збережено: {report_path}")
print("=" * 80)


## 13. Чек-лист згенерованих файлів


In [ ]:
print("\n" + "=" * 80)
print("ЧЕК-ЛИСТ ЗГЕНЕРОВАНИХ ФАЙЛІВ")
print("=" * 80)

expected_files = [
    'synthetic_data.csv',
    'normality_summary.csv',
    'correlations_pairwise.csv',
    'correlations_listwise.csv',
    'correlation_matrix.csv',
    'correlation_matrix.png',
    'scatter_sleep_hours_vs_productivity_score.png',
    'scatter_steps_per_day_vs_weight_kg.png',
    'scatter_calories_intake_vs_weight_kg.png',
    'scatter_caffeine_mg_vs_productivity_score.png',
    'main_pair_conclusion.txt',
    'report_draft.md'
]

print(f"\nПапка: {output_dir.absolute()}\n")

for i, filename in enumerate(expected_files, 1):
    filepath = output_dir / filename
    status = "[OK]" if filepath.exists() else "[MISS]"
    size = filepath.stat().st_size if filepath.exists() else 0
    size_kb = size / 1024
    
    print(f"{i:2d}. {status} {filename:50s} ({size_kb:>8.2f} KB)")

print("\n" + "=" * 80)
print("АНАЛІЗ ЗАВЕРШЕНО УСПІШНО!")
print("=" * 80)
print(f"\nВсього згенеровано {len([f for f in expected_files if (output_dir / f).exists()])} файлів")
print(f"Повний звіт доступний у файлі: {output_dir / 'report_draft.md'}")
print("\nДля перегляду результатів відкрийте папку 'outputs/'")
